In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

First load the model

In [ ]:
# x = Flatten()(inception.output)
# prediction = Dense(CLASS, activation='softmax')(x)
# model = Model(inputs=inception.input, outputs=prediction)


In [ ]:
IMAGE_SIZE = [224,224]
CLASS = 3
inception = InceptionV3(input_shape =IMAGE_SIZE+[3],weights = 'imagenet',include_top = False)  # include_top=False → Removes the last fully connected (dense) layers, so we can add our own layers.
# Pre trained layers will not updated,we will just add new layer with this
for layer in inception.layers:
    layer.trainable = False
# print('Featrure',inception.output)


# Now add the new layers
# The Flatten() layer converts the extracted features into a 1D vector.
x = Flatten()(inception.output)
# print(x)
prediction_layers = Dense(CLASS,activation='softmax')(x)


# print(inception.input)

model = Model(inputs = inception.input,outputs =prediction_layers)
model.compile(
    loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)
model.summary()



Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_658 (Conv2D)       │ (None, 111, 111, 32)   │            864 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_658   │ (None, 111, 111, 32)   │             96 │ conv2d_658[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_658            │ (None, 111, 111, 32)   │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_659 (Conv2D)       │ (None, 109, 109, 32)   │          9,216 │ activation_658[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_659   │ (None, 109, 109, 32)   │             96 │ conv2d_659[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_659            │ (None, 109, 109, 32)   │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_660 (Conv2D)       │ (None, 109, 109, 64)   │         18,432 │ activation_659[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_660   │ (None, 109, 109, 64)   │            192 │ conv2d_660[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_660            │ (None, 109, 109, 64)   │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_28          │ (None, 54, 54, 64)     │              0 │ activation_660[0][0]   │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_661 (Conv2D)       │ (None, 54, 54, 80)     │          5,120 │ max_pooling2d_28[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_661   │ (None, 54, 54, 80)     │            240 │ conv2d_661[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_661            │ (None, 54, 54, 80)     │              0 │ batch_normalization_6… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_662 (Conv2D)  

 Total params: 21,956,387 (83.76 MB)

 Trainable params: 153,603 (600.01 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)

In [ ]:
# Training Data Generator (Without Augmentation)
train_datagen = ImageDataGenerator(rescale=1./255)

# Validation Data Generator (Same as Before)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load Training Data
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/GNNgallbladerDataset/train',
                                                 target_size=(224, 224),
                                                 batch_size=128,
                                                 class_mode='categorical')
# Load Validation Data
val_set = val_datagen.flow_from_directory('/content/drive/MyDrive/GNNgallbladerDataset/test',
                                          target_size=(224, 224),
                                          batch_size=32,
                                          class_mode='categorical')

# Print Optimizer Config
print("\n\n")
model.optimizer.get_config()


In [ ]:
filepath = "/content/drive/MyDrive/SaifulGNNModelOut/inception/highest_val2.h5"
filepath2 = "/content/drive/MyDrive/SaifulGNNModelOut/inception/highest_train2.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='accuracy', verbose=1,
                             save_best_only=True, mode='max')
callbacks_list = [checkpoint1,checkpoint2]
r = model.fit_generator(
    training_set,
    epochs=100,
    validation_data=val_set,
    steps_per_epoch=len(training_set),
    validation_steps=len(val_set),
    callbacks=callbacks_list
)
model.save_weights("/content/drive/MyDrive/SaifulGNNModelOut/inception/end2.h5")

In [ ]:

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.title('Training and validation Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
model.load_weights('/content/drive/MyDrive/SaifulGNNModelOut/inception/highest_val2.h5')
preds = model.evaluate_generator(val_set)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import numpy as np

# Load the weights from the highest validation accuracy model
model.load_weights('/content/drive/MyDrive/SaifulGNNModelOut/inception/highest_val2.h5')

# Make predictions on the validation set
predictions = model.predict_generator(val_set)

# Convert predicted probabilities to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = val_set.classes

# Calculate precision, recall, f1, and confusion matrix
precision, recall, f1, support = precision_recall_fscore_support(true_classes, predicted_classes, average='weighted')
confusion_matrix_result = confusion_matrix(true_classes, predicted_classes)

print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)
print("Confusion Matrix:")
print(confusion_matrix_result)
